使用特定初始化是让反向传播时，传播前和传播后，某层方差保持不变，从而缓解梯度爆炸和梯度消失的问题

普通的sigmoid函数的平均值为0.5并非0，导致神经元输出的方差将大于输入的方差


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
epochs = 1

In [ ]:
# Glorot 和 He 初始化
[name for name in dir(keras.initializers) if not name.startswith('_')]

|初始化|激活函数|$\sigma^2$|
|-|-|-|
|Glorot|None,tanh,logistic,softmax|$\frac{1}{fan_{avg}}$|
|He|ReLU和变体|$\frac{2}{fan_{in}}$|
|LeCun|SELU|$\frac{1}{fan_{in}}$|

In [ ]:
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

In [ ]:
# 如果使用均匀分布但基于fan_avg而不是fan_in
# 进行He初始化，则可以使用Variance_Scaling初始化
init = keras.initializers.VarianceScaling(scale=2, mode='fan_avg'
                                , distribution='uniform')
        
keras.layers.Dense(10, activation='relu', kernel_initializer=init)

相比于ReLU，leaky ReLU能够防止加权和为负数时，神经元只会输出0导致神经元死亡
$$ReLU(z)=\max(0,z) $$

$$leakyReLU(z)=\max(\alpha z,z) $$

后续的，还有随机泄露ReLU（RReLU），在训练过程中在给定范围内随机选择 $\alpha $

有参数化泄露ReLU（PReLU）， $\alpha $将在训练期间学习而不作为超参数

还有一种激活函数，指数线性单位（Exponential Linear Unit，ELU）

$$
ELU(z)=
\begin{cases}

\alpha(e^z - 1)\quad if\quad z<0\\
z\quad if\quad z\geq 0


\end{cases}
$$
如果 $\alpha=1 $则函数所有位置都是平滑的，有助于梯度加速下降

ELU的主要缺点时它的计算比ReLU及变体要慢

SELU 可以使得网路是自归一化的，但是有条件p298

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=epochs,
                    validation_data=(X_valid, y_valid))

* Batch Normaliztion批量归一化
  
    更换激活函数成功缓解了训练一开始时的梯度爆炸、消失，而无法保证训练一段时间后的旧疾复发

    批量归一化通过让模型学习各层输入的最佳缩放和均值，从而缓解训练一段时间后的梯度爆炸、消失
* 大致方法

    在隐藏层的激活函数前，通过该层输入的均值和标准差的 **移动平均值**来估计训练期间的最终统计信息

    1. 通过反向传播想学习输出缩放向量$\gamma $和输出偏移向量$\beta $
    2. 学习使用指数移动平均值估计的最终的输入均值向量$\mu $和最终输入标准差向量$\sigma $
* 具体实现

    使用API实现非常简单直观，只需在每个隐藏层的激活函数之前或之后添加一个批量归一化的层

    但是BN论文的作者主张在激活函数之前添加批量归一化层

    首先是在激活后使用BN，随后是在激活前使用BN

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    # 可选地在模型的第一层后添加一个BN
    keras.layers.BatchNormalization(), 
    # 在激活函数后添加BN 
    keras.layers.Dense(300, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
# 每个BN层的每个输入添加了四个参数
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
            optimizer=keras.optimizers.SGD(learning_rate=1e-3),
            metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=epochs,
                    validation_data=(X_valid, y_valid))


现在试试在激活之前使用BN层

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    # 可选地在模型的第一层之后添加一个BN层
    keras.layers.BatchNormalization(), 
    # 在激活函数之前使用BN， 
    # 在BN层之前的层不需要偏置项，因为BN自带偏置项
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=epochs,
                    validation_data=(X_valid, y_valid))

在反向传播期间裁剪梯度，让它们永远不会超过某个阈值，是另一个缓解梯度爆炸的流行技术

keras中仅仅需要设置clipvalue或clipnorm参数，即可实现梯度裁剪

该优化器会将梯度向量的每个分量都蔡建伟-1.0到1.0之间的值，即将所有损失的偏导数限制在这个范围之间，阈值是可以调整的超参数

应当通过设置clipnorm而不是clipvalue按照范数裁剪，确保梯度裁剪不会改变梯度的方向（按值裁剪、按范数裁剪）

In [ ]:
from pickletools import optimize


optimizer = keras.optimizers.SGD(clipvalue=1.0)
optimizer = keras.optimizers.SGD(clipnorm=1.0)

用keras进行迁移学习

In [ ]:
# 分割fashion Mnist数据集，返回两个训练集
# A训练集 X-标签不是凉鞋且不是衬衫的所有图片，y-X的标签列（多分类器）
# B训练集 X-标签是凉鞋或衬衫的所有图片，y-X的标签是否为衬衫的标签列（二元分类器）
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))
# 按类别分离Fashion Mnist的三大集合
(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [ ]:
X_train_A.shape

In [ ]:
X_train_B.shape

In [ ]:
y_train_A[:30]

In [ ]:
y_train_B[:30]

In [ ]:
# 组件模型A，编译并训练
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [ ]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [ ]:
history = model_A.fit(X_train_A, y_train_A, epochs=epochs,
                    validation_data=(X_valid_A, y_valid_A))

In [ ]:
model_A.save("my_model_A.h5")

In [ ]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
# 参考模型A的架构并训练一个新模型 
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=['accuracy'])

In [ ]:
history = model_B.fit(X_train_B, y_train_B, epochs=epochs,
                      validation_data=(X_valid_B, y_valid_B))

In [ ]:
model_B.summary()

任务B与任务A非常相似，上面的模型B通过模仿模型A的架构来完成训练

现在也可以通过迁移学习来完成

In [ ]:
model_A = keras.models.load_model("my_model_A.h5")
# 获取除了输出层之前的所有层架构及其训练好的权重
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

model_A.layers[:-1]

现在model_A和model_B_on_A共享一些层，对后者的训练也会影响前者的层权重

如果想要分离控制二者，则必须使用api，clone_model()

In [ ]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

由于新的输出层是随机初始化的，可能存在较大的错误梯度，可能会破坏重用模型A的现成权重

* 冻结训练

    为了避免以上情况，就是在训练的前几个伦茨冻结重用的层，给新层一些时间来学习合理的权重
* 代码

    将每一层的可训练属性设置为False并编译模型

In [ ]:
# 冻结重用的前几层（此处是除了是输出层以外的所有层）
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss='binary_crossentropy',
                    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                    metrics=["accuracy"])

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=epochs,
                            validation_data=(X_valid_B, y_valid_B))
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                    metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=epochs,
                            validation_data=(X_valid_B, y_valid_B))



In [ ]:
model_B.evaluate(X_test_B, y_test_B)

In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)

In [ ]:
(100 - 98.65) / (100 - 99.15)

测试的精度高达98.58%, 错误率也降了1.59倍

这么做实际上是“折磨数据直到信服位置”

当论文看起来过于优秀时，你应该要怀疑：也许这个浮华的新技术实际上并没有多大的帮助（事实上，它甚至可能降低性能），但作者尝试了许多变体，仅报告了最好的结果（这可能是由于运气所致），而没有提及他们在途中遇到了多少次失败。

在大多数情况下，这根本不是恶意的，但这是造成如此多科学结果永远无法复现的部分原因

事实上，迁移学习在小型密集网络上不能很好地工作，可能因为小型网络学习的模式很少，密集网络学习的是非常特定的模式，在其他任务中不是很有用

迁移学习最适合深度卷积神经网络，该神经网络倾向于学习更为通用的特征检测器

更快的优化器

<font color='red'>公式中所有更新的变量均为向量</font>
* 动量优化

    动量优化非常关心先前的梯度（加速度概念），每次迭代都会从动量向量m中减去局部梯度，并通过添加该动量向量来更新权重

    为了模拟摩擦机制并防止动量变得过大，引入超参数$\beta $，另外$\eta $还是学习率

    $$m\leftarrow\beta m-\eta\nabla_{\theta}J(\theta) $$
    $$\theta\leftarrow\theta+m $$
* Nesterov加速梯度

    又称NNesterov动量优化，是动量优化的小变体，它不再局部位置$\theta $，而在$\theta+\beta m处沿动量方向稍微提前处测量成本函数的梯度 $

    $$m\leftarrow\beta m-\eta\nabla_{\theta}J(\theta+\beta m) $$
    $$\theta\leftarrow\theta+m $$

    这种小的调整有效是因为通常动量向量会指向正确的方向，因此使用在该方向上测得的更远的梯度而不是原始位置上的梯度会稍微准确一些

* AdaGrad

    通过沿着最陡峭的维度按比例缩小梯度向量

    此处是逐个元素相乘，相除
    $$s\leftarrow s+ \nabla_{\theta}J(\theta)\times\nabla_{\theta}J(\theta)$$
    $$\theta\leftarrow\theta-\eta\nabla_{\theta}J(\theta) / \sqrt{s+\epsilon} $$

    1. 将梯度的平方累计到向量s中
    2. 梯度向量按比例因子$\sqrt{s+\epsilon} $缩小了，其中$\epsilon是平滑项 $
    
    *该算法会降低学习率*，但是对于陡峭的维度，它的执行速度比对缓慢下降的维度的执行速度要快。这称为自适应学习率。好处是算法几乎不需要调整学习率超参数$\eta$
* RMSProp
    
    AdaGrad有可能下降太快，从而永远不能收敛到全局最优解。RMSProp解决了这个问题。

    其通过只是累加最近迭代中的梯度（而不是自训练依赖的所有梯度）来解决这个问题

    $$s\leftarrow \beta s+ (1-\beta)\nabla_{\theta}J(\theta)\times\nabla_{\theta}J(\theta)$$
    $$\theta\leftarrow\theta-\eta\nabla_{\theta}J(\theta) / \sqrt{s+\epsilon} $$

    衰减率$\beta$是一个新的超参数，设置为0.9,通过施加衰减率相关的加权和，强调了最近迭代的梯度，弱化了梯度平方的权重从而避免像AdaGrad那样下降太快

* Adam

    结合了动量优化和RMSProp的思想，代表着自适应矩估计,(逐个元素相乘、逐个元素相除)
    
    1. $$m\leftarrow\beta_1 m-(1-\beta_1)\nabla_{\theta}J(\theta) $$ 
    
    2. $$s\leftarrow \beta_2s+ (1-\beta_2)\nabla_{\theta}J(\theta)\times\nabla_{\theta}J(\theta)$$
    3. $$\hat m\leftarrow\frac{m}{1-\beta^t_1} $$
    4. $$\hat s\leftarrow\frac{s}{1-\beta_2^t} $$
    5. $$\theta\leftarrow\theta-\eta\hat m / \sqrt{s+\epsilon} $$

    1、2、5与动量优化和RMSProp非常相似
    3、4由于m和s初始化为0，因此在训练开始时它们会偏向0，这两个个步骤有助于在训练开始时提高m和s

    Adam也是一种自适应学习率算法，因此对学习率超参数$\eta$需要较少的调整

* Nadam
  
    是在Adam优化上加上了Nesterov技巧，因此其收敛速度通常比Adam快

**优化器比较P317**

**训练稀疏模型**

在训练时使用强l1正规化，迫使优化器产生尽可能多的为0的权重

In [ ]:
# 实现动量优化, 0.9是一个动量可参考的超参值
from re import T


optimizer = keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9)
# 实现Nesterov动量优化
optimizer = keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9, nesterov=True)
# 不应该用AdaGrad
optimizer = keras.optimizers.Adagrad(learning_rate=0.001)
# 使用RMSProp
optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
# 使用Adam beta_1和beta_2的参数是实用的，并非随意给出
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
# 使用Nadam
optimizer = keras.optimizers.Nadam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999)

学习率调度P318
* 幂调度

    $$\eta(t)=\frac{\eta_0}{(1+\frac{t}{s})^c} $$
    $$\eta_0是初始学习率 \quad t是迭代次数$$
    $$s是每s步学习率下降一次，第i次下降将下降到\frac{\eta_0}{i} $$
* 指数调度
    $$\eta(t)= \eta_0 0.1^{\frac{t}{s}}$$
* 分段恒定调度
* 性能调度
* 1周期调度

In [ ]:
# 使用幂调度 decay参数是s的倒数
optimizer = keras.optimizers.SGD(learning_rate=0.01, decay=1e-4)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:

history = model.fit(X_train_scaled, y_train, epochs=epochs,
                    validation_data=(X_valid_scaled, y_valid))

通过正则化避免过拟合 $\ell_1$ and $\ell_2$ regularization

In [ ]:
# l2返回一个正则化函数，
# 在训练过程中的每个步骤都将调用该正则化函数来计算正则化损失
layer = keras.layers.Dense(100, activation='elu',
                           kernel_initializer='he_normal',
                           kernel_regularizer=keras.regularizers.l2(0.01))


In [ ]:
# 通过partial来简化函数内参数的书写
from functools import partial

RegularizedDense = partial(keras.layers.Dense, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer='nadam', metrics=['accuracy'])
history = model.fit(X_train_scaled, y_train, epochs=2,
                    validation_data=(X_valid_scaled, y_valid))


* Dropout技术

    训练过程中，每个神经元都有p概率被删除

    选取了<https://www.zhihu.com/people/li-kai-36-50-82>在<https://zhuanlan.zhihu.com/p/38200980>的评论
    
    说一下我的理解。首先假设一层神经网络中有n个神经元，其中一个神经元的输出是x，输出期望也是x。加上dropout后，有p的概率这个神经元失活，那么这个神经元的输出期望就变成了$(1-p)* x+p * 0=(1-p)x$，我们需要保证这个神经元在训练和测试阶段的输出期望基本不变。那么就有两种方式来解决：
    
        第一种在训练(按概率抛弃部分神经元)的时候，让这个神经元的输出缩放1/(1-p)倍，那么它的输出期望就变成(1-p)x/(1-p)+p*=x，和不dropout的输出期望一致；
    
        第二种方式是在测试(包含所有神经元)的时候，让神经元的输出缩放(1-p)倍，那么它的输出期望就变成了(1-p)x，和训练时的期望是一致的。
    
    

   

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

* alpha dropout

    他是dropout的一种变体，它保留了其输入的均值和标准差（常规的dropout会破坏自归一化）

* MC dropout

    该技术可以提高任何训练后的dropout模型的性能，提供更好地不确定性估计，而无需重新训练甚至不用修改它

In [ ]:
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

* Max norm

    最大范数正则化，对于每个神经元，它会限制传入连接的权重w，使得$||w||_2\leq r $，r是最大范数超参数

    最大范数正则化不会把正则化损失项加入总体损失函数中，而是在每个训练步骤后计算并判断$||w||_2\leq r $来实现。

    每次训练迭代后，模型的fit()方法会调用由max_norm()返回的对象，将该层的权重传递给该对象，并获得返回的缩放权重，然后替换该层的权重

    max_norm()的axis默认为0，意味着最大范数约束将独立应用于每个神经元的权重向量

In [ ]:
layer = keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal",
                           kernel_constraint=keras.constraints.max_norm(1.))

* 总结

    |超参数|默认值|
    |-|-|
    |内核初始化|He初始化|
    |激活函数|ELU|
    |归一化|浅层不需要、深层BN|
    |正则化|提前停止（l2也行）|
    |优化器|动量优化或RMSProp或Nadam|
    |学习率调度|1周期|

    如果网络是密集层的简单堆叠，则它可以自归一化，你应该使用表11-4中的配置。

    |超参数|默认值|
    |-|-|
    |内核初始化|LeCun初始化|
    |激活函数|SELU|
    |归一化|不需要（自归一化）|
    |正则化|如果需要：alpha dropout|
    |优化器|动量优化或RMSProp或Nadam|
    |学习率调度|1周期|